### Original code with Security Vulnerabulities 

In [ ]:
# !pip install flask_sqlalchemy

In [3]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
import threading

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)

with app.app_context():
  db.create_all()

@app.route('/')
def home():
    return "Welcome to the Security Testing Demo!"

@app.route('/users', methods=['GET'])
def get_users():
    users = User.query.all()
    return jsonify([{"id": user.id, "username": user.username, "password": user.password} for user in users])

@app.route('/user/<int:id>', methods=['GET'])
def get_user(id):
    user = User.query.get(id)
    if user:
        return jsonify({"id": user.id, "username": user.username, "password": user.password})
    return jsonify({"message": "User not found"}), 404

@app.route('/user', methods=['POST'])
def add_user():
    data = request.get_json()
    new_user = User(username=data['username'], password=data['password'])
    db.session.add(new_user)
    db.session.commit()
    return jsonify({"message": "User added successfully"}), 201

@app.route('/user/<int:id>', methods=['PUT'])
def update_user(id):
    data = request.get_json()
    user = User.query.get(id)
    if user:
        user.username = data['username']
        user.password = data['password']
        db.session.commit()
        return jsonify({"message": "User updated successfully"})
    return jsonify({"message": "User not found"}), 404

@app.route('/user/<int:id>', methods=['DELETE'])
def delete_user(id):
    user = User.query.get(id)
    if user:
        db.session.delete(user)
        db.session.commit()
        return jsonify({"message": "User deleted successfully"})
    return jsonify({"message": "User not found"}), 404


threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':8080}).start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.5.186:8080
Press CTRL+C to quit
127.0.0.1 - - [03/Nov/2024 14:46:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Nov/2024 14:46:06] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Nov/2024 14:46:31] "GET /users HTTP/1.1" 200 -
192.168.5.186 - - [03/Nov/2024 14:48:58] "GET / HTTP/1.1" 200 -
192.168.5.186 - - [03/Nov/2024 14:48:58] "GET /favicon.ico HTTP/1.1" 404 -


### Check Security Vulnerabulities and genearte updated code

In [ ]:

# As a Security Expert can you check this code above check for Security Vulnerabilities like SQL injection, passwork leakage and all common type of attacks and prepare a report of what updates are needed to make this code secure.

** Explanation of Changes: **

1. Environment-Based Database URI:
- Used os.getenv to set the database URI dynamically, allowing configuration flexibility between development and production.

2. CSRF Protection:
- Added Flask-WTF CSRF protection. For typical API setups, CSRF can be bypassed, especially if using token-based authentication, but this code demonstrates the setup.

3. Password Hashing and Checking:
- Implemented generate_password_hash for password hashing and check_password_hash (if an authentication feature were added).

4. Marshmallow Validation for Input:
- Used Marshmallow to enforce data validation for inputs in add_user and update_user.

5. Information Leakage Prevention:
- Generalized error messages and removed password data from API responses.

6. Concurrency and Secure Deployment:
- Used app.run() only for development. For production, it’s recommended to use a WSGI server like Gunicorn and to secure the app over HTTPS.

7. Secure Secret Key:
- Added a randomly generated secret key required for CSRF protection and session management.

In [ ]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import CSRFProtect
from werkzeug.security import generate_password_hash, check_password_hash
from marshmallow import Schema, fields, validate, ValidationError
import os

# Initialize the Flask app and CSRF protection
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv("DATABASE_URL", "sqlite:///users.db")  # Use environment variable for database URI
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = os.urandom(24)  # Needed for CSRF protection and sessions
db = SQLAlchemy(app)
csrf = CSRFProtect(app)

# Define a User schema using Marshmallow for input validation
class UserSchema(Schema):
    username = fields.String(required=True, validate=validate.Length(min=3, max=80))
    password = fields.String(required=True, validate=validate.Length(min=6))

user_schema = UserSchema()

# Database model for User
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)  # Stored as hashed password

with app.app_context():
    db.create_all()  # Create the database tables if they don't already exist

@app.route('/')
def home():
    return "Welcome to the Security Testing Demo!"

# Get all users (no passwords returned)
@app.route('/users', methods=['GET'])
def get_users():
    users = User.query.all()
    # Exclude passwords in response for security
    return jsonify([{"id": user.id, "username": user.username} for user in users])

# Get a single user by ID (no password returned)
@app.route('/user/<int:id>', methods=['GET'])
def get_user(id):
    user = User.query.get(id)
    if user:
        # Exclude password in response for security
        return jsonify({"id": user.id, "username": user.username})
    return jsonify({"message": "Operation unsuccessful"}), 404  # Generic error message

# Add a new user with hashed password and input validation
@app.route('/user', methods=['POST'])
@csrf.exempt  # CSRF protection is typically for forms; exempt for API if secured by other means
def add_user():
    try:
        # Validate the incoming JSON data
        data = user_schema.load(request.get_json())
    except ValidationError as err:
        return jsonify(err.messages), 400

    # Hash the password for secure storage
    hashed_password = generate_password_hash(data['password'], method='pbkdf2:sha256')
    new_user = User(username=data['username'], password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({"message": "User added successfully"}), 201

# Update an existing user with input validation and hashed password
@app.route('/user/<int:id>', methods=['PUT'])
def update_user(id):
    user = User.query.get(id)
    if not user:
        return jsonify({"message": "Operation unsuccessful"}), 404

    try:
        # Validate the incoming JSON data
        data = user_schema.load(request.get_json())
    except ValidationError as err:
        return jsonify(err.messages), 400

    # Update fields with validated input and hash the new password
    user.username = data['username']
    user.password = generate_password_hash(data['password'], method='pbkdf2:sha256')
    db.session.commit()
    return jsonify({"message": "User updated successfully"})

# Delete a user
@app.route('/user/<int:id>', methods=['DELETE'])
def delete_user(id):
    user = User.query.get(id)
    if not user:
        return jsonify({"message": "Operation unsuccessful"}), 404
    
    db.session.delete(user)
    db.session.commit()
    return jsonify({"message": "User deleted successfully"})

# Ensure the application runs with proper security configurations for production
if __name__ == "__main__":
    # Only use Flask's development server for local testing
    # For production, use a WSGI server (e.g., Gunicorn) and HTTPS
    app.run(host='0.0.0.0', port=8080)


In [ ]:
# !curl -X GET http://172.28.0.12:5000/users
!curl -X GET http://127.0.0.1:8080

In [ ]:
!curl -X GET http://localhost:8080/user/1

In [ ]:
!curl -X POST http://localhost:8080/user -H "Content-Type: application/json" -d '{"username":"testuser","password":"securepassword"}'


In [ ]:
!curl -X PUT http://localhost:8080/user/1 -H "Content-Type: application/json" -d '{"username":"testuser","password":"newsecurepassword"}'

In [ ]:
!curl -X DELETE http://localhost:8080/user/1